In [90]:
import pandas as pd
import sqlite3
import numpy as np
con = sqlite3.connect("natural_disasters.db")
df = pd.read_sql_query('SELECT * FROM natural_disasters_data', con)
df.dropna(axis=1,inplace=True)
df.head()
df.columns

Index(['Year', 'Seq', 'Glide', 'Disaster_Group', 'Disaster_Subgroup',
       'Disaster_Type', 'Disaster_Subtype', 'Country', 'ISO', 'Region',
       'Continent', 'Origin', 'Associated_Dis', 'Associated_Dis2',
       'Aid_Contribution', 'Dis_Mag_Value', 'Dis_Mag_Scale', 'Latitude',
       'Longitude', 'Local_Time', 'Start_Year', 'Start_Month', 'Start_Day',
       'End_Year', 'End_Month', 'End_Day', 'Total_Deaths', 'No_Injured',
       'No_Affected', 'No_Homeless', 'Total_Affected', 'Insured_Damages_USD',
       'Total_Damages_USD', 'CPI'],
      dtype='object')

In [91]:
df_1 = df[['Year','Country','Disaster_Type','Continent','Latitude','Longitude','Total_Deaths','Seq']]
df_1.head()

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28


In [92]:
df_1['Latitude'].astype('Float64')
df_1['Longitude'].astype('Float64')

0        142.007
1          13.98
2       -768.801
3          360.0
4         355.05
          ...   
16113        0.0
16114        0.0
16115        0.0
16116        0.0
16117        0.0
Name: Longitude, Length: 16118, dtype: Float64

In [93]:

df_1 = df_1[~((df['Latitude'] == 0.000) | (df['Longitude'] == 0.000))]
df_1

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28
...,...,...,...,...,...,...,...,...
2717,1991,Costa Rica,Earthquake,Americas,9.685,-83.073,47,114
2718,1991,Panama,Earthquake,Americas,9.685,-83.073,30,114
2719,1990,Costa Rica,Earthquake,Americas,9.869,84.302,1,170
2720,1990,Indonesia,Earthquake,Asia,-9.904,101.120,,702


In [94]:
df_1['Total_Deaths'] = df_1['Total_Deaths'].fillna(np.nan)
df_1['Total_Deaths']= df_1['Total_Deaths'].replace('','0')
df_1['Total_Deaths'] = df_1['Total_Deaths'].astype('int32')

In [95]:
df_1

,Year,Country,Disaster_Type,Continent,Latitude,Longitude,Total_Deaths,Seq
0,2020,Angola,Flood,Africa,-913.908,142.007,30,173
1,2016,Angola,Flood,Africa,-832.104,13.980,23,131
2,2020,Peru,Flood,Americas,-701.073,-768.801,0,122
3,2016,"Tanzania, United Republic of",Flood,Africa,-689.427,360.000,13,244
4,2016,"Tanzania, United Republic of",Flood,Africa,-476.168,355.050,3,28
...,...,...,...,...,...,...,...,...
2717,1991,Costa Rica,Earthquake,Americas,9.685,-83.073,47,114
2718,1991,Panama,Earthquake,Americas,9.685,-83.073,30,114
2719,1990,Costa Rica,Earthquake,Americas,9.869,84.302,1,170
2720,1990,Indonesia,Earthquake,Asia,-9.904,101.120,0,702


In [96]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2719 entries, 0 to 2721
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           2719 non-null   int64  
 1   Country        2719 non-null   object 
 2   Disaster_Type  2719 non-null   object 
 3   Continent      2719 non-null   object 
 4   Latitude       2719 non-null   float64
 5   Longitude      2719 non-null   float64
 6   Total_Deaths   2719 non-null   int32  
 7   Seq            2719 non-null   int64  
dtypes: float64(2), int32(1), int64(2), object(3)
memory usage: 180.6+ KB


In [105]:
total_death = df_1.groupby(['Continent','Country'])['Total_Deaths'].sum()
total_death_df = pd.DataFrame(total_death)
total_death_df

Total_Deaths
Continent Country                        
Africa    Algeria                    6968
          Angola                      353
          Botswana                      0
          Burkina Faso                 68
          Burundi                     104
...                                   ...
Oceania   Samoa                       148
          Solomon Islands             433
          Tonga                        10
          Vanuatu                      12
          Wallis and Futuna             5

[167 rows x 1 columns]

In [98]:
Frequency = df_1.groupby(['Continent','Disaster_Type'])['Disaster_Type'].count()
Frequency

Continent  Disaster_Type    
Africa     Earthquake            70
           Flood                224
           Landslide              2
           Storm                  6
           Volcanic activity      6
Americas   Earthquake           306
           Flood                219
           Landslide              5
           Storm                 40
           Volcanic activity     43
Asia       Earthquake           880
           Flood                419
           Landslide              8
           Storm                 55
           Volcanic activity     32
Europe     Earthquake           184
           Flood                100
           Storm                  3
           Volcanic activity      7
Oceania    Earthquake            55
           Flood                 32
           Landslide              1
           Storm                 10
           Volcanic activity     12
Name: Disaster_Type, dtype: int64

In [99]:
Geo_map_data= df_1.groupby(['Country','Disaster_Type','Continent','Latitude','Longitude','Total_Deaths','Seq'])['Seq'].sum()
Geo_map_data


Country      Disaster_Type  Continent  Latitude  Longitude  Total_Deaths  Seq
Afghanistan  Earthquake     Asia        33.377   71.317     14            202    202
                                        33.426   69.524     2             436    436
                                        33.825   59.809     5             95      95
                                        34.010   71.400     12            166    166
                                        34.185   70.076     22            156    156
                                                                                ... 
Zambia       Flood          Africa     -12.950   24.370     4             10      10
                                       -10.620   32.590     0             40      40
Zimbabwe     Flood          Africa     -20.780   31.240     24            616    616
                                       -19.041   30.804     26            94      94
                                       -16.660   31.240     3           

In [100]:
#total_death.to_json('output.json',orient='records',indent=4)

In [101]:
#total_death_df.to_json('totaldeath_df.json',orient='records',indent=4)